Logistic Regression Implementation on Heart Disease Data

In [13]:
import pandas as pd
import numpy as np
heart=pd.read_csv('heart_disease.csv')

In [7]:
print(heart.head())

   Unnamed: 0  age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  \
0           1   63    1   1       145   233    1        2      150      0   
1           2   67    1   4       160   286    0        2      108      1   
2           3   67    1   4       120   229    0        2      129      1   
3           4   37    1   3       130   250    0        0      187      0   
4           5   41    0   2       130   204    0        2      172      0   

   oldpeak  slope   ca thal  present  
0      2.3      3  0.0  6.0        0  
1      1.5      2  3.0  3.0        1  
2      2.6      2  2.0  7.0        1  
3      3.5      3  0.0  3.0        0  
4      1.4      1  0.0  3.0        0  


In [8]:
#Check for missing data
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  303 non-null    int64  
 1   age         303 non-null    int64  
 2   sex         303 non-null    int64  
 3   cp          303 non-null    int64  
 4   trestbps    303 non-null    int64  
 5   chol        303 non-null    int64  
 6   fbs         303 non-null    int64  
 7   restecg     303 non-null    int64  
 8   thalach     303 non-null    int64  
 9   exang       303 non-null    int64  
 10  oldpeak     303 non-null    float64
 11  slope       303 non-null    int64  
 12  ca          303 non-null    object 
 13  thal        303 non-null    object 
 14  present     303 non-null    int64  
dtypes: float64(1), int64(12), object(2)
memory usage: 35.6+ KB


Based on documentation, what is the definition of each column: <br>
3 age: age in years <br>
4 sex: sex (1 = male; 0 = female)<br>
9 cp: chest pain type<br>
        -- Value 1: typical angina<br>
        -- Value 2: atypical angina<br>
        -- Value 3: non-anginal pain<br>
        -- Value 4: asymptomatic<br>
10 trestbps: resting blood pressure (in mm Hg on admission to the hospital)<br>
12 chol: serum cholestoral in mg/dl <br>
16 fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)<br>
19 restecg: resting electrocardiographic results<br>
        -- Value 0: normal<br>
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)<br>
        -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria<br>
32 thalach: maximum heart rate achieved <br>
38 exang: exercise induced angina (1 = yes; 0 = no)<br>
40 oldpeak = ST depression induced by exercise relative to rest<br>
41 slope: the slope of the peak exercise ST segment<br>
        -- Value 1: upsloping<br>
        -- Value 2: flat<br>
        -- Value 3: downsloping<br>
44 ca: number of major vessels (0-3) colored by flourosopy<br>
51 thal: 3 = normal; 6 = fixed defect; 7 = reversable defect<br>


We can see some variables have been read in as integers but actually represent categories, so data needs cleaning.

In [9]:
#columns 'ca' and 'thal' were read in as object types despite containing
#numerical values according to the documentation. Let's check the issue
heart['ca'].value_counts()


0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: ca, dtype: int64

In [10]:
heart['thal'].value_counts()

3.0    166
7.0    117
6.0     18
?        2
Name: thal, dtype: int64

In [14]:
#drop extraneous index column
heart=heart.drop(heart.columns[0],axis=1)
#convert identified categorical features to str type
heart['cp']=heart['cp'].map({1:'typical_angina',2:'atypical_angina',3:'non_anginal_pain',4:'asymptomatic'})
heart['restecg']=heart['restecg'].map({0:'normal',1:'STT_abnormal',2:'vent_hyper'})
heart['slope']=heart['slope'].map({1:'up',2:'flat',3:'down'})
heart['thal']=heart['thal'].map({'3.0':'normal','6.0':'fixed_defect','7.0':'reversable_defect','?':np.nan})
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    object 
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    object 
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    object 
 11  ca        303 non-null    object 
 12  thal      301 non-null    object 
 13  present   303 non-null    int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 33.3+ KB


In [16]:
heart['thal'].value_counts(dropna=False)

normal               166
reversable_defect    117
fixed_defect          18
NaN                    2
Name: thal, dtype: int64

In [17]:
heart['ca']=heart['ca'].map({'0.0':0,'1.0':1,'2.0':2,'3.0':3,'?':np.nan})

In [19]:
heart['ca'].value_counts(dropna=False)

0.0    176
1.0     65
2.0     38
3.0     20
NaN      4
Name: ca, dtype: int64

In [20]:
#convert categorical predictors to dummies via one-hot encoding
encoded_heart=pd.get_dummies(heart,columns=['cp','restecg','slope','thal'])
encoded_heart.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,present,...,cp_typical_angina,restecg_STT_abnormal,restecg_normal,restecg_vent_hyper,slope_down,slope_flat,slope_up,thal_fixed_defect,thal_normal,thal_reversable_defect
0,63,1,145,233,1,150,0,2.3,0.0,0,...,1,0,0,1,1,0,0,1,0,0
1,67,1,160,286,0,108,1,1.5,3.0,1,...,0,0,0,1,0,1,0,0,1,0
2,67,1,120,229,0,129,1,2.6,2.0,1,...,0,0,0,1,0,1,0,0,0,1
3,37,1,130,250,0,187,0,3.5,0.0,0,...,0,0,1,0,1,0,0,0,1,0
4,41,0,130,204,0,172,0,1.4,0.0,0,...,0,0,0,1,0,0,1,0,1,0


In [27]:
encoded_heart.shape

(303, 23)

In [25]:
# examine correlations
encoded_heart.corr(method='pearson')

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,present,...,cp_typical_angina,restecg_STT_abnormal,restecg_normal,restecg_vent_hyper,slope_down,slope_flat,slope_up,thal_fixed_defect,thal_normal,thal_reversable_defect
age,1.000000,-0.097542,0.284946,0.208950,0.118530,-0.393806,0.091661,0.203805,0.362605,0.223120,...,0.045438,0.084097,-0.157474,0.138313,0.028487,0.170596,-0.184938,0.062042,-0.129234,0.104902
sex,-0.097542,1.000000,-0.064456,-0.199915,0.047862,-0.048663,0.146201,0.102173,0.093185,0.276816,...,0.089828,-0.106574,-0.009339,0.033676,0.047986,-0.002576,-0.021849,0.142524,-0.381754,0.326284
trestbps,0.284946,-0.064456,1.000000,0.130120,0.175340,-0.045351,0.064762,0.189171,0.098773,0.150825,...,0.149737,0.057995,-0.152203,0.139000,0.120930,0.021638,-0.083165,0.075157,-0.136807,0.106210
chol,0.208950,-0.199915,0.130120,1.000000,0.009841,-0.003432,0.061310,0.046564,0.119000,0.085164,...,-0.053021,0.033691,-0.173748,0.166100,-0.047652,0.043538,-0.019245,-0.095743,0.002944,0.053100
fbs,0.118530,0.047862,0.175340,0.009841,1.000000,-0.007854,0.025665,0.005747,0.145478,0.025264,...,0.055511,-0.048305,-0.063587,0.074634,0.105284,-0.033360,-0.020255,0.091351,-0.086774,0.030953
thalach,-0.393806,-0.048663,-0.045351,-0.003432,-0.007854,1.000000,-0.378103,-0.343085,-0.264246,-0.417167,...,0.079683,-0.120829,0.096625,-0.069061,-0.056191,-0.418573,0.446787,-0.159523,0.293614,-0.214326
exang,0.091661,0.146201,0.064762,0.061310,0.025665,-0.378103,1.000000,0.288223,0.145570,0.431894,...,-0.093384,0.042729,-0.089178,0.079445,0.059253,0.257687,-0.287606,0.063073,-0.328539,0.300223
oldpeak,0.203805,0.102173,0.189171,0.046564,0.005747,-0.343085,0.288223,1.000000,0.295832,0.424510,...,0.086959,0.168172,-0.132567,0.094202,0.394253,0.310986,-0.511356,0.104635,-0.339086,0.302145
ca,0.362605,0.093185,0.098773,0.119000,0.145478,-0.264246,0.145570,0.295832,1.000000,0.460442,...,-0.059834,0.040781,-0.132172,0.122813,-0.029606,0.166531,-0.151212,0.088639,-0.253119,0.225453
present,0.223120,0.276816,0.150825,0.085164,0.025264,-0.417167,0.431894,0.424510,0.460442,1.000000,...,-0.088806,0.067605,-0.175790,0.160400,0.061710,0.355709,-0.386789,0.104864,-0.521016,0.480582


In [107]:
#select some predictors based on correlation with target
predictors=['age','thalach','ca','restecg_normal','restecg_vent_hyper']

In [108]:
#Drop incomplete observations for simplicity
encoded_heart=encoded_heart.dropna()
#Split data into train/test X/y
X=encoded_heart[predictors]
y=encoded_heart['present']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=77)

In [109]:
#Verify that train and test have adequate cases and non-cases
print(sum(y_train==1)/len(y_train),sum(y_train==0)/len(y_train))
print(sum(y_test==1)/len(y_test),sum(y_test==0)/len(y_test))

0.46411483253588515 0.5358851674641149
0.45555555555555555 0.5444444444444444


Not prefectly balanced but acceptable for a small dataset

In [110]:
#Train a logistic regression model
from sklearn.linear_model import LogisticRegression

model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [111]:
#Assess the model on training data only
accuracy=model.score(X_train,y_train)


predictions = model.predict(X_train)
tp = sum((predictions == 1) & (y_train == 1))
fp = sum((predictions == 1) & (y_train == 0))
tn = sum((predictions == 0) & (y_train == 0))
fn = sum((predictions == 0) & (y_train == 1))
sens = tp / (tp + fn)
spec = tn / (tn + fp)

print("Training Accuracy: ", accuracy)
print("Training Sensitivity: ", sens)
print("Training Specificity: ", spec)

Training Accuracy:  0.7799043062200957
Training Sensitivity:  0.7010309278350515
Training Specificity:  0.8482142857142857


Overall the training accuracy was about 78%, the sensitivity was 70%, and the specificity was 85%. Based on these metrics, the model seems to perform better for non-cases.

In [112]:
# Checking in terms of log-odds
for predictor, val  in zip(predictors, model.coef_[0]):
    print(predictor, ":", round(val, 2))

age : -0.06
thalach : -0.04
ca : 1.37
restecg_normal : -0.38
restecg_vent_hyper : 0.31


In [113]:
# Checking in terms of odds
for predictor, val  in zip(predictors, model.coef_[0]):
    print(predictor, ":", round(np.exp(val), 2))

age : 0.94
thalach : 0.96
ca : 3.94
restecg_normal : 0.69
restecg_vent_hyper : 1.37


Higher age and maximum heart rate (thalach) is associated with lower odds of heart disease holding the other predictors constant, but both of these odds ratios are close to 1.
Resting ECG showing probable or definite left ventricular hypertrophy by Estes' criteria and the number of colored vessels are associated with higher odds of heart disease holding the other predictors constant. These increases seem to be moderate and high, respectively (a 37% increase and 294% increase).

In [114]:
#Check model preformance on test data
# Checking the various metrics for the model (test set)
accuracy = model.score(X_test, y_test)

predictions = model.predict(X_test)
tp = sum((predictions == 1) & (y_test == 1))
fp = sum((predictions == 1) & (y_test == 0))
tn = sum((predictions == 0) & (y_test == 0))
fn = sum((predictions == 0) & (y_test == 1))
sens = tp / (tp + fn)
spec = tn / (tn + fp)

print("Test Accuracy: ", accuracy)
print("Test Sensitivity: ", sens)
print("Test Specificity: ", spec)

Test Accuracy:  0.7333333333333333
Test Sensitivity:  0.7317073170731707
Test Specificity:  0.7346938775510204


Accuracy dropped from 78% on training to 73% on test data. Sensitivity rose from 70% to 73%. If the goal is to identify true cases of heart disease, sensitivity increase is good. 
Next steps would be a grid search of hyperparameters or to reevaluate chosen predictors and repeat the analysis.